### Full

In [ ]:
from Generation import code_generation
prompt_file_names = [
    "Generation/humaneval_java.jsonl",
]
model_name = "ammarnasr/codegne-finetuned-the-stack-java-v3-checkpoint-1000"
tokenizer_name = "Salesforce/codegen-350M-mono"
generation_strategy = "ammarnasr/pass_at_100_gen_config"
batch_size = 5
for prompt_file_name in prompt_file_names:
    model_name_short = model_name.split("/")[-1]
    model_name_short = model_name_short.replace("-", "_")
    prompt_file_name_short = prompt_file_name.split("/")[-1]
    prompt_file_name_short = prompt_file_name_short.split(".")[0]
    output_file_name = f"Generation/runs/{model_name_short}_{prompt_file_name_short}.jsonl"

    wandb_project_name = f"{model_name_short}_{prompt_file_name_short}"

    args_dict = {
        "prompts_file_name": prompt_file_name,
        "model_name": model_name,
        "tokenizer_name": tokenizer_name,
        "generation_strategy": generation_strategy,
        "output_file_name": output_file_name,
        "wandb_project_name": wandb_project_name,
        "batch_size": batch_size,
    }
    print(args_dict)

    code_generation.main(args_dict)



### LoRa

In [1]:
from Generation import code_generation
prompt_file_names = [
    "Generation/humaneval_java.jsonl",
]
model_name = "ammarnasr/codegne-finetuned-LoRa-the-stack-java-v5-checkpoint-1000"
tokenizer_name = "Salesforce/codegen-350M-mono"
generation_strategy = "ammarnasr/pass_at_100_gen_config"
batch_size = 10
for prompt_file_name in prompt_file_names:
    model_name_short = model_name.split("/")[-1]
    model_name_short = model_name_short.replace("-", "_")
    prompt_file_name_short = prompt_file_name.split("/")[-1]
    prompt_file_name_short = prompt_file_name_short.split(".")[0]
    output_file_name = f"Generation/runs/{model_name_short}_{prompt_file_name_short}.jsonl"

    wandb_project_name = f"{model_name_short}_{prompt_file_name_short}"

    args_dict = {
        "prompts_file_name": prompt_file_name,
        "model_name": model_name,
        "tokenizer_name": tokenizer_name,
        "generation_strategy": generation_strategy,
        "output_file_name": output_file_name,
        "wandb_project_name": wandb_project_name,
        "batch_size": batch_size,
    }
    print(args_dict)

    code_generation.main(args_dict, with_peft=True)



ModuleNotFoundError: No module named 'Generation'